## Check current river flow at favorite paddling spots 
The output is a map that identifies my favorite New England rivers to paddle with a green icon if there is a paddleable flow rate, red icon if not. The USGS gauge locations and prefered flow rates are read in from a custom csv file (see in repository). Most suggested paddleable flow rates are from [American Whitewater](https://www.americanwhitewater.org/).

In [1]:
import os
import re
import datetime
import pandas as pd
import folium
import dataretrieval.nwis as nwis

In [2]:
def convert2deg(lat_or_long):
    """Converts a latatitude or longitude string to decimal degrees as float 
    Args: a lat or long in deg-min-sec 
        ex: 42°36'43"N 
    Returns: a lat or long in decimal degrees
        ex: 42.6119
    """
    
    LL = lat_or_long.strip()  # strip spaces at ends of string
    deg, minutes, seconds, direction = re.split(r"\D", LL, maxsplit=3)  # split by any nondigit character
    newdd = float(deg) + float(minutes) / 60 + float(seconds) / (60 * 60) # convert to DD
    if direction == "S" or direction == "W":
        newdd = newdd * -1
    else:
        newdd
    newdd = round(newdd, 4)
    return newdd

In [3]:
def read_file(filename):
    """Reads csv file and creates a list
    Args: filename of the csv
    Returns: a nested list of points, each point containing:
        0. tuple of coordinates
        1. location description
        2. USGS water gauge site ID number
        3. minimum discharge to paddle river
        4. maximum discharge to paddle river
        5. other info 
    """
    
    pointsList = pd.read_csv(filename)
    coord = []
    for row in pointsList.iterrows():
        coord.append(
            [
                tuple([convert2deg(row[1].latitude), convert2deg(row[1].longitude)]),
                row[1].location,
                str(row[1].site).rjust(8, "0"),  # rjust maintains the leading zero
                row[1].minflow,
                row[1].maxflow,
                row[1].other,
            ]
        )
    return coord

In [4]:
def accessUSGS(sitenum, date):
    """Retrieves data from USGS's National Water Information System (NWIS)
    From: https://github.com/USGS-python/dataretrieval
    
    Args: USGS site number, todays date in format "YYYY-MM-DD"
    Returns: daily mean value of water discharge in CFS
    """
    
    # gets daily values (dv) for flow rates (includes mean, median, min, max, etc)
    info = nwis.get_record(sites=sitenum, service="dv", start=date, parameterCd="00060") 
    if info.empty:
        discharge = -999  # -999 code for no data available
    else:
        discharge = info["00060_Mean"].values[0]  # 00060 is USGS code for discharge
    return discharge

In [5]:
def addDV(poi_list):
    """Appends the CFS daily value to the points list
    Args: list of points as returned by read_file() function
    Returns: list of points including today's mean CFS info
    """
    
    today = datetime.date.today() #get todays date in format: "YYYY-MM-DD"
    for p in poi_list:
        cfs = accessUSGS(p[2], today) #use USGS site ID number to retrieve flow rate data 
        p.append(float(cfs))  # daily mean CFS stored at p[6]
    return poi_list

In [6]:
def compareRates(poi_list):
    """Compares current CFS to min and max flows for paddling that location
        "green" for within range
        "red" for out of range
    Args: points list as returned by addDV() function
    Returns: points list with new string item "green" or "red" appended 
    """
    
    for p in poi_list:
        if (p[6] >= p[3]) and (p[6] <= p[4]):
            p.append("green")
        else:
            p.append("red")
    return poi_list


In [7]:
def createMap(zoom, poi_list):
    """Builds and formats the map
    Args: the zoom level of initial map, points list as returned by compareRates() function
    Returns: folium map 
    """

    # finds average of points for the center of the map
    lat_view = round((sum(p[0][0] for p in poi_list) / len(poi_list)), 4)
    lon_view = round((sum(p[0][1] for p in poi_list) / len(poi_list)), 4)
    
    today = datetime.date.today() #gets todays date 

    myMap = folium.Map(location=[lat_view, lon_view], zoom_start=zoom)
    # head_metadata must be a separate string to prevent {overflow-y: hidden} from conflicting with f-string.
    head_metadata = "<head><style> html {overflow-y: hidden;} </style></head>"
    title_html = f'{head_metadata} <h3 align="center" style="font-size:20px"><b> Flow Rates for {today}</b></h3>'
    myMap.get_root().html.add_child(folium.Element(title_html)) #access map html and add title in html format 
    for p in poi_list:
        iframe = folium.IFrame(
            html=f"""<b>Location:</b> {p[1]} <br> 
                     <b>Current flow:</b> {str (p[6])} CFS <br> 
                     <b> Paddleable flow range: </b> {p[3]} - {p[4]} CFS
                """,
            width="300px",
            height="100px"
        )
        configIcon = folium.Icon(color=p[7], icon="location-dot")
        folium.Marker(
            location=[p[0][0], p[0][1]],
            popup=folium.map.Popup(iframe, show=False),
            icon=configIcon,
            
        ).add_to(myMap)
    return myMap




filename = "USGS-water-gauge-locations.csv"
pois = read_file(filename)
response = addDV(pois)
final = compareRates(response)
createMap(6, final)